In [38]:
%load_ext autoreload
%autoreload 2

%pylab inline
import numpy as np
import pandas as pd

import seaborn as sns
sns.set_style('ticks')
sns.set_context('paper')
import os
import eternabench as eb


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


In [39]:
package_info

,title,category
package,,
vienna_1,Vienna 1,1
vienna_2,Vienna 2,1
vienna_2_nodangles,"Vienna 2, no dangles",1
vienna_langdon_pars,"Vienna 2, `Langdon 2018` params",1
vienna_rnasoft_pars,"Vienna 2, `RNASoft 2007` params",1
vienna_2_60C,"Vienna 2, 60˚C",1
nupack_95,NUPACK 1995,2
nupack_95_nodangles,"NUPACK 1995, no dangles",2
nupack_99,NUPACK 1999,2


In [60]:
cm = pd.read_csv(os.environ['ETERNABENCH_PATH']+'/scoring_data/backup/EB_-efold_pearson_zscores_by_Dataset.csv')
cm['Kind'] = 'Chem Mapping'
print(len(cm.Dataset.unique()))
cm_agg = cm.groupby('package')['pearson_zscore_by_Dataset_mean'].agg(['mean','std'])
cm_agg.columns = ['cm_mean','cm_std']

rs = pd.read_csv(os.environ['ETERNABENCH_PATH']+'/scoring_data/RS_bps_pearson_zscores.csv')
rs['Kind'] = 'Riboswitch'
print(len(rs.Dataset.unique()))
rs_agg = rs.groupby('package')['pearson_zscore_by_Dataset_mean'].agg(['mean','std'])
rs_agg.columns = ['rs_mean','rs_std']

total_agg = pd.concat([cm_agg, rs_agg],axis=1)

total_agg['net_mean'] = total_agg.apply(lambda row: np.mean([row['rs_mean'], row['cm_mean']]),axis=1)
total_agg['net_std'] = pd.concat([cm, rs],axis=0).groupby('package')['pearson_zscore_by_Dataset_mean'].std()
total_agg = total_agg.sort_values('net_mean',ascending=False)

total_agg['ChemMapping Z-score: mean(SEM)'] = total_agg.apply(lambda row: "%.2f(%.2f)"% (row['cm_mean'], row['cm_std']),axis=1)
total_agg['Riboswitch Z-score: mean(SEM)'] = total_agg.apply(lambda row: "%.2f(%.2f)"% (row['rs_mean'], row['rs_std']),axis=1)
total_agg['Both dataset types: mean(SEM)'] = total_agg.apply(lambda row: "%.2f(%.2f)"% (row['net_mean'], row['net_std']),axis=1)

package_info = eb.plot.get_packages()
total_agg['Package'] = [package_info.loc[x]['title'] for x in total_agg.index]

total_agg.to_csv('SUPP_TABLE_PRECURSORS/table1precursor.csv')

24
12


In [61]:
! open SUPP_TABLE_PRECURSORS/table1precursor.csv

In [39]:
net = pd.concat([cm,rs])
packages = eb.plot.get_packages()

net = net.merge(packages,on='package')

kind_ranking = net.groupby(['title'])['pearson_zscore_by_Dataset_mean'].agg(['mean','std'])
kind_ranking = kind_ranking.sort_values('mean',ascending=False)
for x in kind_ranking.index:
    print(x)

CONTRAfold 2
Vienna 2, 60˚C
RNAsoft BL, no dangles
RNAsoft BLstar
RNAstructure, 60˚C
CONTRAfold 1
RNAsoft LAM-CG
RNAsoft 2007
Vienna 2, `RNASoft 2007` params
CONTRAfold 2, noncomplementary
RNAsoft NOM-CG
Vienna 2, `Langdon 2018` params
Vienna 2
RNAstructure
RNAstructure, no coaxial stacking
RNAsoft 1999, no dangles
RNAsoft 1999
NUPACK 1999
NUPACK 1995
Vienna 2, no dangles
NUPACK 1995, no dangles
NUPACK 1999, no dangles
Vienna 1
NUPACK 1999, 60˚C


In [68]:
zscores = pd.read_csv(os.environ['ETERNABENCH_PATH']+'/scoring_data/EternaBench_external_zscores_by_dataset_MANUAL_COMPILED.csv')
zscores = zscores.loc[zscores.window_size==900]
tmp = zscores.groupby(['Class','package'])['pearson_zscore_by_Dataset_mean'].mean()
tmp = tmp.reset_index()
tmp.pivot(index=['package'], columns='Class').to_csv('table2_precursor_mean.csv')
tmp = zscores.groupby(['Class','package'])['pearson_zscore_by_Dataset_mean'].std()
tmp = tmp.reset_index()
tmp.pivot(index=['package'], columns='Class').to_csv('table2_precursor_stdev.csv')

In [73]:
zscores = pd.read_csv(os.environ['ETERNABENCH_PATH']+'/scoring_data/EternaBench_external_zscores_by_dataset_MANUAL_COMPILED.csv')
zscores = zscores.loc[zscores.window_size==900]
tmp = zscores.groupby(['package'])['pearson_zscore_by_Dataset_mean'].mean()
for x in tmp.values:
    print(x)

tmp = zscores.groupby(['package'])['pearson_zscore_by_Dataset_mean'].std()
for x in tmp.values:
    print(x)

0.3952413186666666
1.2619893468888892
-1.2167268807500002
0.3022213680277777
-1.0128084679444447
0.15620217719444443
-0.9543088229166669
0.12184683133333334
0.40138837764721136
0.38607962747327584
0.2386191978589235
0.33855619185080665
0.3875404533633963
0.4358649352047612
0.36862684083360403
0.37583633184874427
